<a href="https://colab.research.google.com/github/Ahm1027/Payloads-Model/blob/master/Payload_Model_Ngrams.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
import pandas as pd
import numpy as np
import os
import tensorflow as tf
from tensorflow.keras import Sequential
from keras.layers import Activation, Dense, Dropout
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelBinarizer, LabelEncoder
from sklearn import preprocessing
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
from nltk import ngrams
from sklearn.preprocessing import Normalizer
import matplotlib.pyplot as plt
from google.colab import drive
import binascii

In [2]:
drive.mount('/content/drive')

Mounted at /content/drive


In [37]:
def get_hexdump(executable, num_of_ngrams=4):
  executable = open(executable, 'rb')
  data = executable.read()
  executable.close()
  data2hex = bin2hex(data)
  output = ngrams(data2hex.split(" "), num_of_ngrams)
  return [ ''.join(grams) for grams in output]

def bin2hex(data):
  data2hex = ""
  for byte in data:
    hex = str(binascii.hexlify(bytes([byte]))).split("'")
    if (len(hex) >= 2):
      data2hex += hex[1]+" "
    else:
      data2hex += hex[0]+" "
  return data2hex

# def generate_ngrams(text, WordsToCombine):
#     words = text.split()
#     output = []  
#     for i in range(len(words)- WordsToCombine+1):
#       word = ''
#       for i in range (i, i+WordsToCombine):
#         word = word + words[i]
#       output.append(word)
#     return output

# def calculateTotalCounts(uniqueOutput, output):
#   Uniquecounts = []
#   for i in range(len(uniqueOutput)):
#     count = 0
#     for j in range(len(output)):
#       if output[j] == uniqueOutput[i]:
#         count = count+1
#     Uniquecounts.append(count)
#   return Uniquecounts

In [4]:
def vectorize(data):
  vectorizer = CountVectorizer()
  vectorizer.fit(data)
  vectorized = vectorizer.transform(data)
  return vectorizer, vectorized

In [6]:
def arrangeAndSort(data_vectorizer, data_vectorized):
  data_arranged = list(zip(data_vectorizer.get_feature_names(), data_vectorized.sum(0).getA1()))
  data_sorted = sorted(data_arranged, key=lambda item: item[1], reverse=True)
  return data_sorted

In [34]:
def resolveMalwareFiles(FOLDER_PATHS, num_of_features=2000):
  stopCheck, prev=0, 0
  globalFeatures = np.array([])
  for files in os.listdir(FOLDER_PATHS):
    filepath = r"/content/drive/MyDrive/Payloads/"+files
    data = get_hexdump(filepath)
    return data
    data_vectorizer, data_vectorized = vectorize(data)
    data_sorted = arrangeAndSort(data_vectorizer, data_vectorized)[:num_of_features]
    if (len(globalFeatures) == 0):
      globalFeatures = np.array(data_sorted)[:, 0].astype(str).tolist()
    else:
      globalFeatures = np.append(globalFeatures, np.array(data_sorted)[:, 0].astype(str).tolist())
      indexes = np.unique(globalFeatures, return_index=True)[1]
      globalFeatures = [globalFeatures[index] for index in sorted(indexes)]
    if (stopCheck == prev+10):
      prev = stopCheck
      print("%f percent completed"%((stopCheck/1000)*100), end=", ")
      print("Number of Features: %d"%(len(globalFeatures)))
    stopCheck += 1
  return globalFeatures

globalFeatures = resolveMalwareFiles(r"/content/drive/MyDrive/Payloads")
print(len(globalFeatures))

688721


In [ ]:
x = pd.HDFStore("some_file.hdf")

x.append("a", pd.DataFrame(globalFeatures))
x.close()

my_data = pandas.HDFStore("some_file.hdf")
usable_a_copy = my_data["a"]

copy_as_nparray = usable_a_copy.values

In [ ]:
globalFeatures.tofile('features.dat')

In [ ]:
np.save('features_np.npy', globalFeatures)

In [ ]:
def normalizeFeatures(data):
  data_scaled = (data - data.min()) / (data.max() - data.min())   #MinMax Normalization
  return data_scaled

In [ ]:
def generateGlobalVocab(data):
  globalFeatures = np.array([])
  for key, value in data:
    l = np.array(value)[:,0]
    globalFeatures = np.concatenate([globalFeatures, l], axis=0)

  globalFeatures

def find_first_no_jit(item, vec):
  for i in range(vec.size):
    if item == vec[i]:
      return i
  return -1

def mergeWithGlobalVocab(data, globalFeatures):
  if globalFeatures.size == 0:
    globalFeatures = np.concatenate([globalFeatures, np.array(data)[:, 0].astype(str).tolist()])
    print(globalFeatures.shape)
  else:
    #globalFeatures[0] = np.concatenate([globalFeatures, np.array(data)[:, 0].astype(str).tolist()])
    print(globalFeatures.shape)
    globalFeatures[0] = np.append(globalFeatures[0], [globalFeatures, np.array(data)[:, 0].astype(str).tolist()])
    print(len(globalFeatures[0]))
    globalFeatures[0], indexes = np.unique(globalFeatures, return_index=True)
    globalFeatures[0] = globalFeatures[np.sort(indexes)]
  print(globalFeatures)
  features = [0]*globalFeatures.size
  for x in data:
    id = find_first_no_jit(x[0], globalFeatures)
    if (id != -1):
      features[id] = x[1]
  globalFeatures = np.vstack((globalFeatures, features))
  return globalFeatures

(2, 2000)

In [ ]:
# compressed_SVD = TruncatedSVD(30)
# compressed_data = compressed_SVD.fit_transform(features_frequency)
# pd.DataFrame(compressed_SVD.components_)